# OrientDB tutorial

## Prerequisites

### Documentation

You will find all documentation for :
* [OrientDB SQL reference](http://www.orientdb.com/docs/last/SQL-Functions.html)
* [Orientdb python client](http://orientdb.com/docs/last/PyOrient-Client.html#working-with-the-client)

## Import libraries

In [1]:
import pyorient

In [2]:
ROOT_PASSWORD = "root"
client = pyorient.OrientDB("orientdb", 2424)
session_id = client.connect("root", ROOT_PASSWORD)

In [3]:
print(client.db_list())

{{'databases': {}}}


## I. Quick start

### Creating the database

**Q:** Create a database `gods` as a `GRAPH_DATABASE` in `MEMORY_STORAGE_TYPE`. 

We will use it to store relationships between Greek deities.

In [4]:
gods = client.db_create("gods", pyorient.DB_TYPE_GRAPH, pyorient.STORAGE_TYPE_MEMORY)

**Q:** Connect your pyorient client to the `gods` database.

In [5]:
client.db_open("gods","root", ROOT_PASSWORD)

**Q:** You should now be able to launch OrientDB queries through the Python client with the [command()](http://orientdb.com/docs/last/PyOrient-Client-Command.html) function. 

You should think of OrientDB as a Graph-Document database for the following questions. Each vertex and edge will contain information on it inside a JSON document.

Create a new Vertex with content `{name: 'Zeus', symbol: 'thunder'}`. The [CREATE VERTEX : Create a vertex using JSON content](http://orientdb.com/docs/last/SQL-Create-Vertex.html) doc page should help you.

In [6]:
client.command("create vertex content {name: 'Zeus', symbol: 'thunder'}")

You have created a VERTEX in the previous question. The VERTEX is a [class](https://orientdb.com/docs/last/Tutorial-Classes.html) of OrientDB which defines a record that can be linked to others through EDGE instances.

You can find all VERTEX created in the database with a SQL command on the `V` table, like `SELECT * FROM V`. 

**Q:** Print all current vertices in `gods`, it should only have `Zeus` though for now.

In [7]:
for elem in client.command("SELECT * FROM V"):
    print(elem)

{'@V':{'name': 'Zeus', 'symbol': 'thunder'},'version':1,'rid':'#9:0'}


**Q:** Create new vertices with content : 
```
{name:Héra, symbol:tiara}
{name:Poséidon, symbol:trident}
{name:Athena, symbol:helmet}
{name:Arès, symbol:weapons} 
```

In [8]:
client.command("create vertex content {name: 'Héra', symbol: 'tiara'}")
client.command("create vertex content {name: 'Poséidon', symbol: 'trident'}")
client.command("create vertex content {name: 'Athéna', symbol: 'helmet'}")
client.command("create vertex content {name: 'Arès', symbol: 'weapons'}")

**Q:** Display all vertices with name = `Arès`

In [9]:
for elem in client.command("SELECT * FROM V where name='Arès'"):
    print(elem)

{'@V':{'name': 'Arès', 'symbol': 'weapons'},'version':1,'rid':'#13:0'}


**Q:** Create an EDGE from `Zeus` to `Poséidon` with the content `{kind: 'sibling'}

In [10]:
client.command("CREATE edge FROM (SELECT FROM V where name='Zeus') TO (SELECT FROM V where name='Poséidon') CONTENT {'kind':'sibling'}")

**Q:** Redisplay all vertices, discuss.

In [11]:
for elem in client.command("SELECT * FROM V"):
    print(elem)

{'@V':{'name': 'Zeus', 'symbol': 'thunder', 'out_': <pyorient.otypes.OrientBinaryObject object at 0x7f3e383b0530>},'version':2,'rid':'#9:0'}
{'@V':{'name': 'Héra', 'symbol': 'tiara'},'version':1,'rid':'#10:0'}
{'@V':{'name': 'Poséidon', 'symbol': 'trident', 'in_': <pyorient.otypes.OrientBinaryObject object at 0x7f3e383b0920>},'version':2,'rid':'#11:0'}
{'@V':{'name': 'Athéna', 'symbol': 'helmet'},'version':1,'rid':'#12:0'}
{'@V':{'name': 'Arès', 'symbol': 'weapons'},'version':1,'rid':'#13:0'}


**Q:** Display all edges. They are contained in the class `E`

In [12]:
for elem in client.command("SELECT * FROM E"): # expand(out)
    print(elem.oRecordData)

{'kind': 'sibling', 'out': <pyorient.otypes.OrientRecordLink object at 0x7f3e383b07a0>, 'in': <pyorient.otypes.OrientRecordLink object at 0x7f3e383b0890>}


Two fields on vertices have appeared, containing the outgoing (out_) and incoming (in_) links.

At the edge level, two fields point to the original (out) and destination (in) vertices.

**Q:** Lets create some more edges :

* Zeus > Héra (sibling)
* Zeus > Arès (father)
* Zeus > Athena (father)
* Héra > Arès (mother)
* Héra > Zeus (sibling)
* Poséidon > Zeus (sibling)

_Hint 1 :_ check [the CREATE EDGE doc page](http://orientdb.com/docs/last/SQL-Create-Edge.html) to find an example for creating edges on vertices using subqueries so you can run queries to fetch the vertices before creating an edge in between.

_Hint 2 :_ after you have found the command to create edges between vertices with sub-queries, you should be well-versed enough in Python to create a list of all edges in the question, and loop the command on each element of the list to create all edges in one go =)

In [13]:
froms = ["Zeus", "Zeus", "Zeus", "Héra", "Héra", "Poséidon"]
tos = ["Héra", "Arès", "Athéna", "Arès", "Zeus", "Zeus"]
kinds = ["sibling", "father", "father", "mother", "sibling", "sibling"]
for fromg, tog, kindg in zip(froms, tos, kinds):
    client.command(f"CREATE edge FROM (SELECT FROM V where name='{fromg}') TO (SELECT FROM V where name='{tog}') CONTENT {{'kind':'{kindg}'}}")

### Looking for data

**Q:** Using [out()](http://orientdb.com/docs/last/Tutorial-Working-with-graphs.html#querying-graphs) function, display all vertices connected and outgoing from Zeus.

You should use the EXPAND() special function to transform the vertex collection in the result-set by expanding it, making the results more readable.

In [14]:
for elem in client.command("SELECT expand(out()) FROM V where name='Zeus'"):
    print(elem)

{'@V':{'name': 'Poséidon', 'symbol': 'trident', 'in_': <pyorient.otypes.OrientBinaryObject object at 0x7f3e383997f0>, 'out_': <pyorient.otypes.OrientBinaryObject object at 0x7f3e383994f0>},'version':3,'rid':'#11:0'}
{'@V':{'out_': <pyorient.otypes.OrientBinaryObject object at 0x7f3e3839be00>, 'name': 'Héra', 'symbol': 'tiara', 'in_': <pyorient.otypes.OrientBinaryObject object at 0x7f3e3839bd40>},'version':4,'rid':'#10:0'}
{'@V':{'in_': <pyorient.otypes.OrientBinaryObject object at 0x7f3e3839be30>, 'name': 'Arès', 'symbol': 'weapons'},'version':3,'rid':'#13:0'}
{'@V':{'name': 'Athéna', 'symbol': 'helmet', 'in_': <pyorient.otypes.OrientBinaryObject object at 0x7f3e383b07d0>},'version':2,'rid':'#12:0'}


**Q:** Display all vertices which got a father (the vertices which are the destination of an arc whose kind attribute is father).

_Hint: You can notice that we use the field `in` the arc, and not the function `in()` which applies to vertices._

In [15]:
for elem in client.command("SELECT expand(in) FROM E where kind='father'"):
    print(elem)

{'@V':{'in_': <pyorient.otypes.OrientBinaryObject object at 0x7f3e38398530>, 'name': 'Arès', 'symbol': 'weapons'},'version':3,'rid':'#13:0'}
{'@V':{'name': 'Athéna', 'symbol': 'helmet', 'in_': <pyorient.otypes.OrientBinaryObject object at 0x7f3e38499cd0>},'version':2,'rid':'#12:0'}


**Q:** As in SQL, the operator `in` used in a clause `where` allows to restrict the possible values with an embedded query _(where ... in (select ...))_. 

Display the mothers, by displaying the vertices where an outgoing arc is part of the arcs where kind is a mother.

In [16]:
for elem in client.command("SELECT * FROM V WHERE name in (SELECT name from (SELECT expand(out) FROM E WHERE kind='mother'))"):
    print(elem)

{'@V':{'name': 'Héra', 'out_': <pyorient.otypes.OrientBinaryObject object at 0x7f3e3837f980>, 'symbol': 'tiara', 'in_': <pyorient.otypes.OrientBinaryObject object at 0x7f3e387893a0>},'version':4,'rid':'#10:0'}


**Q:** Display the brothers and sisters of Zeus (the destination summits of an arc whose kind is sibling and whose original summit is Zeus).

In [17]:
for elem in client.command("SELECT * FROM (SELECT * FROM V WHERE name in (SELECT name from (SELECT expand(out) FROM E WHERE kind='sibling'))) WHERE name NOT IN 'Zeus'"):
    print(elem)

{'@V':{'name': 'Héra', 'out_': <pyorient.otypes.OrientBinaryObject object at 0x7f3e383997f0>, 'symbol': 'tiara', 'in_': <pyorient.otypes.OrientBinaryObject object at 0x7f3e38523d10>},'version':4,'rid':'#10:0'}
{'@V':{'name': 'Poséidon', 'symbol': 'trident', 'in_': <pyorient.otypes.OrientBinaryObject object at 0x7f3e383b0d70>, 'out_': <pyorient.otypes.OrientBinaryObject object at 0x7f3e383b0f50>},'version':3,'rid':'#11:0'}


## Modeling a Product Recommendation System

You are currently modeling the data of a product recommendation system with OrientDB.

The main purpose of such a system is to answer the question "which products were purchased by their people who purchased product X? »

Purchased products have only one name field. They are purchased by people who have a nickname.

When a person buys a product, the date of purchase is stored. 

Instead of working with "anonymous" vertices and arcs, you will use classes. The `create class` command allows you to create custom classes.

The vertex classes must extend V, the arc classes must extend E.

**Q:** Create an `eCommerce` database, and the necessary classes to model the system.

PS : you can view all classes in the database with :

```python
for name in client.command("SELECT name FROM (SELECT expand(classes) FROM metadata:schema)"):
    print(name)
```

In [18]:
client.db_create("eCommerce", pyorient.DB_TYPE_GRAPH, pyorient.STORAGE_TYPE_MEMORY)
client.db_open("eCommerce","root", ROOT_PASSWORD)

In [19]:
# Vertices
client.command("CREATE CLASS Product EXTENDS V")
client.command("CREATE PROPERTY Product.name STRING")
client.command("CREATE INDEX Product.name ON Product (name) UNIQUE")
client.command("CREATE CLASS Person EXTENDS V")
client.command("CREATE PROPERTY Person.nickname STRING")
client.command("CREATE INDEX Person.nickname ON Person (nickname) UNIQUE")

# Edges
client.command("CREATE CLASS Purchased EXTENDS E")
client.command("CREATE PROPERTY Purchased.date STRING")

[1]

In [20]:
for name in client.command("SELECT name FROM (SELECT expand(classes) FROM metadata:schema)"):
    print(name)

{{'name': 'Product'},'version':0,'rid':'#-2:0'}
{{'name': 'V'},'version':0,'rid':'#-2:1'}
{{'name': 'Purchased'},'version':0,'rid':'#-2:2'}
{{'name': 'E'},'version':0,'rid':'#-2:3'}
{{'name': 'OIdentity'},'version':0,'rid':'#-2:4'}
{{'name': 'OFunction'},'version':0,'rid':'#-2:5'}
{{'name': 'OUser'},'version':0,'rid':'#-2:6'}
{{'name': 'ORestricted'},'version':0,'rid':'#-2:7'}
{{'name': 'Person'},'version':0,'rid':'#-2:8'}
{{'name': 'OSchedule'},'version':0,'rid':'#-2:9'}
{{'name': 'OTriggered'},'version':0,'rid':'#-2:10'}
{{'name': 'ORole'},'version':0,'rid':'#-2:11'}
{{'name': 'OSequence'},'version':0,'rid':'#-2:12'}


**Q:** Create the following products: `spaghetti`, `bolognese sauce`, `cheese`, `apple`.

In [21]:
prods = ["spaghetti", "bolognese sauce", "cheese", "apple"]
for prod in prods:
    client.command(f"CREATE VERTEX Product CONTENT {{'name': '{prod}'}}")

for prod in client.command("SELECT * FROM Product"):
    print(prod.oRecordData)

{'name': 'spaghetti'}
{'name': 'bolognese sauce'}
{'name': 'cheese'}
{'name': 'apple'}


**Q:** Create the following people: `peter`, `meredith`.

In [22]:
persons = ["peter", "meredith"]
for pers in persons:
    client.command(f"CREATE VERTEX Person CONTENT {{'nickname': '{pers}'}}")

# Affichage des personnes
for pers in client.command("SELECT * FROM Person"):
    print(pers.oRecordData)

{'nickname': 'peter'}
{'nickname': 'meredith'}


**Q:** Create the following purchases: 
- peter > spaghetti + cheese on 20/01/2016 
- meredith > cheese + apple + bolognese sauce on 22/01/2016
- peter > spaghetti + bolognese sauce on 27/01/2016


In [23]:
purchases = [
    {'person': 'peter', 'products': ['spaghetti', 'cheese'], 'date': '2016-01-20'},
    {'person': 'meredith', 'products': ['cheese', 'apple', 'bolognese sauce'], 'date': '2016-01-22'},
    {'person': 'peter', 'products': ['spaghetti', 'bolognese sauce'], 'date': '2016-01-27'}
]

for purch in purchases:
    person = purch['person']
    products = purch['products']
    date = purch['date']
    for product in products:
        client.command(f"CREATE EDGE Purchased FROM (SELECT FROM Person WHERE nickname = '{person}') TO (SELECT FROM Product WHERE name = '{product}') CONTENT {{'date': '{date}'}}")

for purchase in client.command("SELECT * FROM Purchased"):
    print(purchase.oRecordData)

{'date': '2016-01-20', 'out': <pyorient.otypes.OrientRecordLink object at 0x7f3e3839bd40>, 'in': <pyorient.otypes.OrientRecordLink object at 0x7f3e383b2270>}
{'date': '2016-01-20', 'out': <pyorient.otypes.OrientRecordLink object at 0x7f3e383b2240>, 'in': <pyorient.otypes.OrientRecordLink object at 0x7f3e383b2540>}
{'date': '2016-01-22', 'out': <pyorient.otypes.OrientRecordLink object at 0x7f3e383b1c70>, 'in': <pyorient.otypes.OrientRecordLink object at 0x7f3e383b28a0>}
{'date': '2016-01-22', 'out': <pyorient.otypes.OrientRecordLink object at 0x7f3e383b2ab0>, 'in': <pyorient.otypes.OrientRecordLink object at 0x7f3e383b2ba0>}
{'date': '2016-01-22', 'out': <pyorient.otypes.OrientRecordLink object at 0x7f3e383b2de0>, 'in': <pyorient.otypes.OrientRecordLink object at 0x7f3e383b2ed0>}
{'date': '2016-01-27', 'out': <pyorient.otypes.OrientRecordLink object at 0x7f3e383b3080>, 'in': <pyorient.otypes.OrientRecordLink object at 0x7f3e383b3170>}
{'date': '2016-01-27', 'out': <pyorient.otypes.Orien

**Q:** Who bought Bolognese sauce?

In [ ]:
query = "SELECT nickname FROM Person WHERE OUT('Purchased') IN (SELECT FROM Product WHERE name = 'bolognese sauce')"
request = client.command(query)

for pers in request:
    print(pers.oRecordData)

{'nickname': 'peter'}
{'nickname': 'meredith'}


**Q:** It is possible to link the `out` and `in` navigation functions. What products are purchased with Bolognese sauce? 

In [ ]:
query = """
SELECT DISTINCT(name) 
FROM (
  SELECT expand(out('Purchased')) AS product
  FROM Person
  WHERE @rid IN (
    SELECT expand(in('Purchased')) 
    FROM Product 
    WHERE name = 'bolognese sauce'
  )
)
WHERE name <> 'bolognese sauce'
"""
results = client.command(query)

for product in results:
    print(product.oRecordData)

{'DISTINCT': 'spaghetti'}
{'DISTINCT': 'cheese'}
{'DISTINCT': 'apple'}


## Postquisites

Since we create databases in memory, they get destroyed on server shutdown.